In [40]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,lit

In [2]:
spark = SparkSession.builder \
        .master("local[*]") \
        .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/27 09:29:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [28]:
df_green = spark.read.parquet('data/raw/green/*/*')
df_yellow = spark.read.parquet('data/raw/yellow/*/*')

In [29]:
df_green = df_green.withColumn("VendorID", col("VendorID").cast("int")) \
                    .withColumn("RatecodeID", col("RatecodeID").cast("int")) \
                    .withColumn("PULocationID", col("PULocationID").cast("int")) \
                    .withColumn("DOLocationID", col("DOLocationID").cast("int")) \
                    .withColumn("passenger_count", col("passenger_count").cast("int")) \
                    .withColumn("ehail_fee", col("ehail_fee").cast("double")) \
                    .withColumn("payment_type", col("payment_type").cast("int")) \
                    .withColumn("trip_type", col("trip_type").cast("int"))

df_yellow = df_yellow.withColumn("VendorID", col("VendorID").cast("int")) \
                    .withColumn("passenger_count", col("passenger_count").cast("int")) \
                    .withColumn("RatecodeID", col("RatecodeID").cast("int")) \
                    .withColumn("DOLocationID", col("DOLocationID").cast("int")) \
                    .withColumn("PULocationID", col("PULocationID").cast("int")) \
                    .withColumn("payment_type", col("payment_type").cast("int")) \
                    .withColumn("airport_fee", col("airport_fee").cast("double"))

In [30]:
df_green.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- lpep_pickup_datetime: timestamp (nullable = true)
 |-- lpep_dropoff_datetime: timestamp (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- ehail_fee: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- trip_type: integer (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



In [31]:
df_yellow.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)



In [33]:
df_yellow = df_yellow.withColumnRenamed('tpep_pickup_datetime','pickup_datetime') \
                        .withColumnRenamed('tpep_dropoff_datetime','dropoff_datetime')

df_green = df_green.withColumnRenamed('lpep_pickup_datetime','pickup_datetime') \
                        .withColumnRenamed('lpep_dropoff_datetime','dropoff_datetime')

In [34]:
set(df_green.columns) & set(df_yellow.columns)

{'DOLocationID',
 'PULocationID',
 'RatecodeID',
 'VendorID',
 'congestion_surcharge',
 'dropoff_datetime',
 'extra',
 'fare_amount',
 'improvement_surcharge',
 'mta_tax',
 'passenger_count',
 'payment_type',
 'pickup_datetime',
 'store_and_fwd_flag',
 'tip_amount',
 'tolls_amount',
 'total_amount',
 'trip_distance'}

In [35]:
common_columns = []

for col in df_green.columns:
    if col in df_yellow.columns:
        common_columns.append(col)
common_columns

['VendorID',
 'pickup_datetime',
 'dropoff_datetime',
 'store_and_fwd_flag',
 'RatecodeID',
 'PULocationID',
 'DOLocationID',
 'passenger_count',
 'trip_distance',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'payment_type',
 'congestion_surcharge']

In [42]:
df_green_common = df_green.select(common_columns).withColumn('service_type', lit('green'))
df_yellow_common = df_yellow.select(common_columns).withColumn('service_type', lit('yellow'))

In [43]:
df_trips_data = df_green_common.unionAll(df_yellow_common)

In [44]:
df_trips_data.groupBy('service_type').count().show()

+------------+--------+
|service_type|   count|
+------------+--------+
|       green| 2802931|
|      yellow|55553400|
+------------+--------+



In [48]:
df_trips_data.createOrReplaceTempView('trips_data')

In [52]:
spark.sql("""
SELECT 
    service_TYPE, 
    COUNT(1) 
FROM trips_data 
GROUP BY
    service_type
""").show()

+------------+--------+
|service_TYPE|count(1)|
+------------+--------+
|       green| 2802931|
|      yellow|55553400|
+------------+--------+



In [58]:
df_result = spark.sql("""
SELECT 
    -- Reveneue grouping 
    PULocationID AS revenue_zone,
    date_trunc('month', pickup_datetime) AS revenue_month, 
    service_type, 
    -- Revenue calculation 
    SUM(fare_amount) AS revenue_monthly_fare,
    SUM(extra) AS revenue_monthly_extra,
    SUM(mta_tax) AS revenue_monthly_mta_tax,
    SUM(tip_amount) AS revenue_monthly_tip_amount,
    SUM(tolls_amount) AS revenue_monthly_tolls_amount,
    SUM(improvement_surcharge) AS revenue_monthly_improvement_surcharge,
    SUM(total_amount) AS revenue_monthly_total_amount,
    SUM(congestion_surcharge) AS revenue_monthly_congestion_surcharge,
    -- Additional calculations
    AVG(passenger_count) AS avg_montly_passenger_count,
    AVG(trip_distance) AS avg_montly_trip_distance
FROM
    trips_data
GROUP BY
    1, 2, 3
""")

In [59]:
df_result.show()

+------------+-------------------+------------+--------------------+---------------------+-----------------------+--------------------------+----------------------------+-------------------------------------+----------------------------+------------------------------------+--------------------------+------------------------+
|revenue_zone|      revenue_month|service_type|revenue_monthly_fare|revenue_monthly_extra|revenue_monthly_mta_tax|revenue_monthly_tip_amount|revenue_monthly_tolls_amount|revenue_monthly_improvement_surcharge|revenue_monthly_total_amount|revenue_monthly_congestion_surcharge|avg_montly_passenger_count|avg_montly_trip_distance|
+------------+-------------------+------------+--------------------+---------------------+-----------------------+--------------------------+----------------------------+-------------------------------------+----------------------------+------------------------------------+--------------------------+------------------------+
|          42|2019-

In [60]:
df_result.write.parquet('data/report/revenue')